<a href="https://colab.research.google.com/github/Anshika91/Deep_Learning/blob/main/TrainingLoop_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
import numpy as np

In [ ]:
tf.constant(np.arange(10))

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Normalize pixel values to the range of 0-1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


4422102/4422102 [==============================] - 0s 0us/step


In [3]:
def create_model():
    model = keras.Sequential([
        layers.Reshape(target_shape=(28, 28, 1), input_shape=(28, 28)),
        layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(units=64, activation='relu'),
        layers.Dense(units=10, activation='softmax')
    ])
    return model

model = create_model()

In [4]:
# Define the loss function and optimizer
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer_upper = keras.optimizers.Adam(learning_rate=0.001)  # Learning rate for upper layers
optimizer_lower = keras.optimizers.Adam(learning_rate=0.0001)  # Learning rate for lower layers

# Metrics to track the mean training loss and accuracy
train_loss_metric = keras.metrics.Mean()
train_accuracy_metric = keras.metrics.SparseCategoricalAccuracy()

# Metrics to track the validation loss and accuracy
val_loss_metric = keras.metrics.Mean()
val_accuracy_metric = keras.metrics.SparseCategoricalAccuracy()

# Define the batch size and number of epochs
batch_size = 64
epochs = 10

# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    
    # Reset the metrics
    train_loss_metric.reset_states()
    train_accuracy_metric.reset_states()
    val_loss_metric.reset_states()
    val_accuracy_metric.reset_states()
    
    # Iterate over the batches of the training data
    for step, (x_batch_train, y_batch_train) in enumerate(
        tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
    ):
        with tf.GradientTape() as tape:
            # Forward pass
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        gradients = tape.gradient(loss_value, model.trainable_variables)
        
        # Apply different learning rates to upper and lower layers
        gradients_upper = gradients[:4]
        gradients_lower = gradients[4:]
        
        # Update weights using the respective optimizers
        optimizer_upper.apply_gradients(zip(gradients_upper, model.trainable_variables[:4]))
        optimizer_lower.apply_gradients(zip(gradients_lower, model.trainable_variables[4:]))
        
        # Update the training metrics
        train_loss_metric.update_state(loss_value)
        train_accuracy_metric.update_state(y_batch_train, logits)
        
        # Print training progress
        if step % 200 == 0:
            print(
                f"Iteration {step}/{len(x_train)//batch_size} - "
                f"Mean Loss: {train_loss_metric.result():.4f} - "
                f"Mean Accuracy: {train_accuracy_metric.result() * 100:.2f}%"
            )
    
    # Perform validation at the end of each epoch
    val_logits = model(x_val, training=False)
    val_loss_value = loss_fn(y_val, val_logits)
    
    # Update the validation metrics
    val_loss_metric.update_state(val_loss_value)
    val_accuracy_metric.update_state(y_val, val_logits)
    
    # Print validation results
    print(
        f"Validation Loss: {val_loss_metric.result():.4f} - "
        f"Validation Accuracy: {val_accuracy_metric.result() * 100:.2f}%\n"
    )

Epoch 1/10
Iteration 0/750 - Mean Loss: 2.2850 - Mean Accuracy: 7.81%
Iteration 200/750 - Mean Loss: 0.6821 - Mean Accuracy: 77.25%
Iteration 400/750 - Mean Loss: 0.5616 - Mean Accuracy: 80.82%
Iteration 600/750 - Mean Loss: 0.5084 - Mean Accuracy: 82.50%
Validation Loss: 0.3538 - Validation Accuracy: 87.98%

Epoch 2/10
Iteration 0/750 - Mean Loss: 0.3506 - Mean Accuracy: 89.06%
Iteration 200/750 - Mean Loss: 0.3435 - Mean Accuracy: 88.11%
Iteration 400/750 - Mean Loss: 0.3272 - Mean Accuracy: 88.53%
Iteration 600/750 - Mean Loss: 0.3241 - Mean Accuracy: 88.63%
Validation Loss: 0.3078 - Validation Accuracy: 89.33%

Epoch 3/10
Iteration 0/750 - Mean Loss: 0.2651 - Mean Accuracy: 89.06%
Iteration 200/750 - Mean Loss: 0.2895 - Mean Accuracy: 90.04%
Iteration 400/750 - Mean Loss: 0.2786 - Mean Accuracy: 90.22%
Iteration 600/750 - Mean Loss: 0.2786 - Mean Accuracy: 90.18%
Validation Loss: 0.2897 - Validation Accuracy: 89.81%

Epoch 4/10
Iteration 0/750 - Mean Loss: 0.2159 - Mean Accuracy: 9